# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f62f8c19a30>
├── 'a' --> tensor([[-0.2258, -1.3123, -1.5467],
│                   [-0.2269,  0.0207, -0.8492]])
└── 'x' --> <FastTreeValue 0x7f62f8c19760>
    └── 'c' --> tensor([[-0.6626,  1.3912, -1.0609, -0.1021],
                        [ 0.5688,  1.4765,  0.1690, -0.6049],
                        [-0.0909, -0.1654,  0.4285, -0.7030]])

In [4]:
t.a

tensor([[-0.2258, -1.3123, -1.5467],
        [-0.2269,  0.0207, -0.8492]])

In [5]:
%timeit t.a

62.8 ns ± 0.0366 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f62f8c19a30>
├── 'a' --> tensor([[-0.3808, -0.2203,  1.4867],
│                   [-0.5364, -2.0935,  0.4359]])
└── 'x' --> <FastTreeValue 0x7f62f8c19760>
    └── 'c' --> tensor([[-0.6626,  1.3912, -1.0609, -0.1021],
                        [ 0.5688,  1.4765,  0.1690, -0.6049],
                        [-0.0909, -0.1654,  0.4285, -0.7030]])

In [7]:
%timeit t.a = new_value

66.4 ns ± 0.0567 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2258, -1.3123, -1.5467],
               [-0.2269,  0.0207, -0.8492]]),
    x: Batch(
           c: tensor([[-0.6626,  1.3912, -1.0609, -0.1021],
                      [ 0.5688,  1.4765,  0.1690, -0.6049],
                      [-0.0909, -0.1654,  0.4285, -0.7030]]),
       ),
)

In [10]:
b.a

tensor([[-0.2258, -1.3123, -1.5467],
        [-0.2269,  0.0207, -0.8492]])

In [11]:
%timeit b.a

52.2 ns ± 0.049 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.3514, -0.5151,  0.6412],
               [ 0.9653,  0.6915, -1.7599]]),
    x: Batch(
           c: tensor([[-0.6626,  1.3912, -1.0609, -0.1021],
                      [ 0.5688,  1.4765,  0.1690, -0.6049],
                      [-0.0909, -0.1654,  0.4285, -0.7030]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.054 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

946 ns ± 13.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 39.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 2.41 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 348 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f624c919430>
├── 'a' --> tensor([[[-0.2258, -1.3123, -1.5467],
│                    [-0.2269,  0.0207, -0.8492]],
│           
│                   [[-0.2258, -1.3123, -1.5467],
│                    [-0.2269,  0.0207, -0.8492]],
│           
│                   [[-0.2258, -1.3123, -1.5467],
│                    [-0.2269,  0.0207, -0.8492]],
│           
│                   [[-0.2258, -1.3123, -1.5467],
│                    [-0.2269,  0.0207, -0.8492]],
│           
│                   [[-0.2258, -1.3123, -1.5467],
│                    [-0.2269,  0.0207, -0.8492]],
│           
│                   [[-0.2258, -1.3123, -1.5467],
│                    [-0.2269,  0.0207, -0.8492]],
│           
│                   [[-0.2258, -1.3123, -1.5467],
│                    [-0.2269,  0.0207, -0.8492]],
│           
│                   [[-0.2258, -1.3123, -1.5467],
│                    [-0.2269,  0.0207, -0.8492]]])
└── 'x' --> <FastTreeValue 0x7f624c9196a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.5 µs ± 55.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f624d00a070>
├── 'a' --> tensor([[-0.2258, -1.3123, -1.5467],
│                   [-0.2269,  0.0207, -0.8492],
│                   [-0.2258, -1.3123, -1.5467],
│                   [-0.2269,  0.0207, -0.8492],
│                   [-0.2258, -1.3123, -1.5467],
│                   [-0.2269,  0.0207, -0.8492],
│                   [-0.2258, -1.3123, -1.5467],
│                   [-0.2269,  0.0207, -0.8492],
│                   [-0.2258, -1.3123, -1.5467],
│                   [-0.2269,  0.0207, -0.8492],
│                   [-0.2258, -1.3123, -1.5467],
│                   [-0.2269,  0.0207, -0.8492],
│                   [-0.2258, -1.3123, -1.5467],
│                   [-0.2269,  0.0207, -0.8492],
│                   [-0.2258, -1.3123, -1.5467],
│                   [-0.2269,  0.0207, -0.8492]])
└── 'x' --> <FastTreeValue 0x7f62f8c19b20>
    └── 'c' --> tensor([[-0.6626,  1.3912, -1.0609, -0.1021],
                        [ 0.5688,  1.4765,  0.1690, -0.6049],
                 

In [23]:
%timeit t_cat(trees)

29.6 µs ± 88.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.8 µs ± 68.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.2258, -1.3123, -1.5467],
                [-0.2269,  0.0207, -0.8492]],
       
               [[-0.2258, -1.3123, -1.5467],
                [-0.2269,  0.0207, -0.8492]],
       
               [[-0.2258, -1.3123, -1.5467],
                [-0.2269,  0.0207, -0.8492]],
       
               [[-0.2258, -1.3123, -1.5467],
                [-0.2269,  0.0207, -0.8492]],
       
               [[-0.2258, -1.3123, -1.5467],
                [-0.2269,  0.0207, -0.8492]],
       
               [[-0.2258, -1.3123, -1.5467],
                [-0.2269,  0.0207, -0.8492]],
       
               [[-0.2258, -1.3123, -1.5467],
                [-0.2269,  0.0207, -0.8492]],
       
               [[-0.2258, -1.3123, -1.5467],
                [-0.2269,  0.0207, -0.8492]]]),
    x: Batch(
           c: tensor([[[-0.6626,  1.3912, -1.0609, -0.1021],
                       [ 0.5688,  1.4765,  0.1690, -0.6049],
                       [-0.0909, -0.1654,  0.4285, -0.7030]],
         

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 262 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.2258, -1.3123, -1.5467],
               [-0.2269,  0.0207, -0.8492],
               [-0.2258, -1.3123, -1.5467],
               [-0.2269,  0.0207, -0.8492],
               [-0.2258, -1.3123, -1.5467],
               [-0.2269,  0.0207, -0.8492],
               [-0.2258, -1.3123, -1.5467],
               [-0.2269,  0.0207, -0.8492],
               [-0.2258, -1.3123, -1.5467],
               [-0.2269,  0.0207, -0.8492],
               [-0.2258, -1.3123, -1.5467],
               [-0.2269,  0.0207, -0.8492],
               [-0.2258, -1.3123, -1.5467],
               [-0.2269,  0.0207, -0.8492],
               [-0.2258, -1.3123, -1.5467],
               [-0.2269,  0.0207, -0.8492]]),
    x: Batch(
           c: tensor([[-0.6626,  1.3912, -1.0609, -0.1021],
                      [ 0.5688,  1.4765,  0.1690, -0.6049],
                      [-0.0909, -0.1654,  0.4285, -0.7030],
                      [-0.6626,  1.3912, -1.0609, -0.1021],
                      [ 0.5688,  

In [28]:
%timeit Batch.cat(batches)

140 µs ± 326 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

311 µs ± 850 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
